<a href="https://colab.research.google.com/github/MuajiiTsai/110-1-NTU-DBME5028/blob/main/DeepLearning_MidTerm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn import metrics
from PIL import Image
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from google.colab import drive

model_path = "model.pt"

##Connect Google Drive

In [ ]:
drive.mount('/content/drive')
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
os.chdir(train_path)
training_data_file = os.listdir(train_path)
print(training_data_file[0][:-11])
type(training_data_file[0])
# os.chdir(test_path)
# testing_data_file = os.listdir(test_path)
# print(len(training_data_file))

Mounted at /content/drive
WRIST_cfde832782


str

##Dataset

In [ ]:
from torchvision.io import read_image, ImageReadMode
#train_dataset

Resize = transforms.Compose([
  transforms.Resize([512,512])
])


class BoneDataset(Dataset):
    def __init__(self, annotations_file, directory, transform=None, target_transform=None):
        self.dir = directory
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = [os.path.join(directory, f) for f in os.listdir(directory)]
        self.transform = transform
        self.target_transform = target_transform
        
        # for idx in range(4000):

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        # Get patient ID
        ID = self.img_labels.iloc[idx, 0]
        patient_image = torch.tensor([])
        i = 0
        for f in os.listdir(self.dir):
          # If file name equals ID
          if patient_image.size(dim=0) == 4:
            break
          else:
            if f[:-11] == ID:
              img_path = self.img_dir[i]
              image = Resize(read_image(img_path, mode=ImageReadMode.GRAY))
              patient_image = torch.cat((patient_image, image), 0)
            i = i + 1
        # print(f'patient{idx}: {patient_image.size(dim=0)}')
        # print(patient_image.size(dim=0))
        while patient_image.size(dim=0) < 4:
          if (4 - patient_image.size(dim=0)) == 1:
            image = transforms.functional.vflip(patient_image[2])
            torch.reshape(image, (1,image.shape[0], image.shape[1]))
            patient_image = torch.cat((patient_image, image), 0)
          elif (4 - patient_image.size(dim=0)) == 2:
            image = transforms.functional.hflip(patient_image[1])
            patient_image = torch.cat((patient_image, image), 0)
          else:
            image = transforms.functional.vflip(patient_image[0])
            patient_image = torch.cat((patient_image, image), 0)  
        # img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])        
        patient_image = patient_image.type(torch.FloatTensor)
        # img_path = self.img_dir[idx]
        # image = read_image(img_path, mode=ImageReadMode.RGB)
        # image = image.type(torch.FloatTensor)
        # label = self.img_labels.iloc[idx, 1]
        # if self.transform:
        #     image = self.transform(image)
        # if self.target_transform:
        #     label = self.target_transform(label)
        # return image, label
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            patient_image = self.transform(patient_image)
        if self.target_transform:
            label = self.target_transform(label)
        return patient_image, label

In [ ]:
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
test_path = '/content/drive/My Drive/Deep_Learning/Midterm/test/'
label_file = '/content/drive/My Drive/Deep_Learning/Midterm/train.csv'

#model
batch_size = 32
learning_rate = 1e-4
num_show = 3
transform = transforms.Compose([
  transforms.Grayscale(num_output_channels=1),
  # transforms.PILToTensor(),
  transforms.Resize([512,512]),
  # transforms.ToPILImage(mode='RGB')
])



train_dataset = BoneDataset(label_file, train_path, transform=transform)
# test_dataset = ...


train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle = True
)
'''
test_dataloader = torch.utils.data.DataLoader(
    #test_dataset,
    batch_size=1,
    shuffle = False
)
'''
model = models.efficientnet_b0(pretrained=True)
# model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(
    1,
    64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False
)
model = nn.Sequential(model, nn.Linear(1000,1))
model = model.cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.SmoothL1Loss()
# print(train_dataloader.dataset)

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"feature batch size: {train_features.size()}")
print(f"labels batch size: {train_labels.size()}")
img = train_features[1].squeeze()
label = train_labels[1]
print(img)
# plt.imshow(img)
# plt.show()
print(f"Label: {label}")

torch.Size([512, 512])


RuntimeError: ignored

In [ ]:
num_epochs = 1

#train the model
best_test_loss = np.inf

for idx_epoch in range(num_epochs):
  print(f"Epoch{idx_epoch}")

  #training phase
  model.train()

  train_losses = []
  for image_batch, label_batch in tqdm(train_dataloader):
    optimizer.zero_grad()
    image_batch = image_batch.cuda()
    label_batch = label_batch.cuda()
    output_batch = model(image_batch)
    # print(output_batch.shape)
    loss = loss_fn(output_batch, label_batch)
    loss.backward()
    optimizer.step()

    loss = loss.detach().cpu().numpy()
    train_losses.append(loss)

  train_loss = np.mean(train_losses)
  print(f"Training loss: {train_loss}")

  # y = np.array([1,1,2,2])
  # y_pred = np.array([0.1,0.4,0.35,2])
  # fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=2)
  # print(metrics.auc(fpr, tpr))

  '''
  #testing phase
  model.eval()

  test_losses = []
  test_accuracies = []
  for image_batch, label_batch in tqdm(test_dataloader):
    image_batch = image_batch.cuda()
    label_batch = label_batch.cuda()
    output_batch = model(image_batch)
    loss = loss_fn(output_batch, label_batch)
    
    loss = loss.detach().cpu().numpy()
    train_losses.append(loss)
    
    ##### AUC #####
    

    accuracy = np.mean(is_correct_batch)
    test_accuracies.append(accuracy)
    
    

    #####
    test_loss = np.mean(test_losses)
    test_accuracy = np.mean(test_accuracies)

    if test_loss < best_test_loss:
      best_test_loss = test_loss
      torch.save(model.state_dict(), model_path)
      print("The model is saved")
    '''
  torch.save(model.state_dict(), model_path)
  print("The model is saved")

Epoch0


  0%|          | 0/125 [00:00<?, ?it/s]

torch.Size([512, 512])


RuntimeError: ignored

#Playground

##Import CSV

In [ ]:
df = pd.read_csv (r'/content/drive/My Drive/Deep_Learning/Midterm/train.csv',dtype=str)
sorted_df = df.sort_values(by='id',kind='mergesort')
# print(df)
print(sorted_df)

NameError: ignored

##Import Data

In [ ]:
train_dataset = torch.tensor([])
transform1 = transforms.Compose([
  transforms.PILToTensor(),
])
print(train_dataset)
for i in range(100):
  img = Image.open(f"{train_path}/{training_data_file[i]}")
  img_tensor = transform1(img)
  a = torch.cat((train_dataset, img_tensor),0)
  # print(a.shape)

tensor([])
torch.Size([1, 512, 379])
torch.Size([3, 512, 379])
torch.Size([1, 512, 278])
torch.Size([1, 512, 329])
torch.Size([1, 512, 278])
torch.Size([1, 512, 384])
torch.Size([3, 512, 406])
torch.Size([1, 512, 221])
torch.Size([3, 512, 406])
torch.Size([3, 512, 425])
torch.Size([3, 512, 406])
torch.Size([1, 512, 306])
torch.Size([1, 512, 349])
torch.Size([1, 512, 286])
torch.Size([1, 512, 223])
torch.Size([3, 512, 405])
torch.Size([1, 512, 246])
torch.Size([1, 512, 286])
torch.Size([1, 512, 215])
torch.Size([3, 406, 512])
torch.Size([3, 512, 424])
torch.Size([3, 406, 512])
torch.Size([1, 512, 258])
torch.Size([1, 512, 273])
torch.Size([1, 507, 512])
torch.Size([3, 512, 406])
torch.Size([3, 512, 405])
torch.Size([3, 512, 406])
torch.Size([3, 512, 379])
torch.Size([3, 512, 406])
torch.Size([1, 512, 242])
torch.Size([1, 512, 409])
torch.Size([3, 512, 406])
torch.Size([3, 512, 406])
torch.Size([1, 512, 372])
torch.Size([1, 512, 325])
torch.Size([3, 512, 407])
torch.Size([3, 512, 405])
t